# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
#import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import mode

# Challenge 1 - The `stats` Submodule

This submodule contains statistical functions for conducting hypothesis tests, producing various distributions and other useful tools. Let's examine this submodule using the KickStarter dataset. Load the data using the `ks-projects-201801.csv` file

In [2]:
ks = pd.read_csv('ks-projects-201801.csv')

Now print the `head` function to examine the dataset.

In [3]:
ks.head()
#ks.shape

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


Import the `mode` function from `scipy.stats` and find the mode of the `country` and `currency` column.

In [4]:
#MODE FROM SCIPY
from scipy.stats import mode
counmode=st.mode(ks['country'])
counmode

ModeResult(mode=array(['US'], dtype=object), count=array([292627]))

In [5]:
currmode=st.mode(ks['currency'])
currmode

ModeResult(mode=array(['USD'], dtype=object), count=array([295365]))

The trimmed mean is a function that computes the mean of the data with observations removed. The most common way to compute a trimmed mean is by specifying a percentage and then removing elements from both ends. However, we can also specify a threshold on both ends. The goal of this function is to create a more robust method of computing the mean that is less influenced by outliers. SciPy contains a function called `tmean` for computing the trimmed mean. 

In the cell below, import the `tmean` function and then find the 75th percentile of the `goal` column. Compute the trimmed mean between 0 and the 75th percentile of the column. Read more about the `tmean` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.tmean.html#scipy.stats.tmean).

In [6]:
#TMEAN FROM SCIPY
from scipy.stats import tmean

meangoal=st.tmean(ks['goal'])
meangoal

49080.79152056851

#### SciPy contains various statistical tests. One of the tests is Fisher's exact test. This test is used for contingency tables. 

The test originates from the "Lady Tasting Tea" experiment. In 1935, Fisher published the results of the experiment in his book. The experiment was based on a claim by Muriel Bristol that she can taste whether tea or milk was first poured into the cup. Fisher devised this test to disprove her claim. The null hypothesis is that the treatments do not affect outcomes, while the alternative hypothesis is that the treatment does affect outcome. To read more about Fisher's exact test, see:

* [Wikipedia's explanation](http://b.link/test61)
* [A cool deep explanation](http://b.link/handbook47)
* [An explanation with some important Fisher's considerations](http://b.link/significance76)

Let's perform Fisher's exact test on our KickStarter data. We intend to test the hypothesis that the choice of currency has an impact on meeting the pledge goal. We'll start by creating two derived columns in our dataframe. The first will contain 1 if the amount of money in `usd_pledged_real` is greater than the amount of money in `usd_goal_real`. We can compute this by using the `np.where` function. If the amount in one column is greater than the other, enter a value of 1, otherwise enter a value of zero. Add this column to the dataframe and name it `goal_met`.

In [7]:
# hypothesis :the choice of currency has an impact on meeting the pledge goal- Fisher's exact test

#1) Creating 2 columns (populate with np.where) :
#col 1
ks["goal_met"]= np.where(ks['usd_pledged_real']>ks['usd_goal_real'],1,0)
ks["goal_met"]

0         0
1         0
2         0
3         0
4         0
         ..
378656    0
378657    0
378658    0
378659    0
378660    0
Name: goal_met, Length: 378661, dtype: int32

Next, create a column that checks whether the currency of the project is in US Dollars. Create a column called `usd` using the `np.where` function where if the currency is US Dollars, assign a value of 1 to the row and 0 otherwise.

In [8]:
#col 2
ks["usd"]= np.where(ks['currency']=='USD',1,0)
ks["usd"]


0         0
1         1
2         1
3         1
4         1
         ..
378656    1
378657    1
378658    1
378659    1
378660    1
Name: usd, Length: 378661, dtype: int32

Now create a contingency table using the `pd.crosstab` function in the cell below to compare the `goal_met` and `usd` columns.

Import the `fisher_exact` function from `scipy.stats` and conduct the hypothesis test on the contingency table that you have generated above. You can read more about the `fisher_exact` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html#scipy.stats.fisher_exact). The output of the function should be the odds ratio and the p-value. The p-value will provide you with the outcome of the test.

In [9]:
from scipy.stats import fisher_exact
#2) contingency table using the pd.crosstab function

conttabl=pd.crosstab(ks["usd"],ks["goal_met"])
conttabl
#Running the hypothesis on the contingency table
oddsr, pvalue = fisher_exact(conttabl)
pvalue
# hypothesis :the choice of currency has an impact on meeting the pledge goal

9.75e-321

# Challenge 2 - The `interpolate` submodule

This submodule allows us to interpolate between two points and create a continuous distribution based on the observed data.

In the cell below, import the `interp1d` function from `scipy.interpolate` and first take a sample of 10 rows from `kickstarter`. 

**Make sure there are not duplicated values in `backers`**

In [28]:
from scipy.interpolate import interp1d

kssample=ks.sample(10)
kssample

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,goal_met,usd
234207,260582055,Cowboy Cricket Kitchen: Edible Insect Kitchen ...,Farms,Food,USD,2017-09-27,15000.0,2017-08-23 18:36:53,3728.0,failed,62,US,1823.00,3728.00,15000.00,0,1
267778,432422474,Startup For Local Farm To Food Truck,Food Trucks,Food,USD,2015-05-15,8000.0,2015-04-15 17:20:47,895.0,failed,15,US,895.00,895.00,8000.00,0,1
273953,463839824,Always: A Marine and an American adventure,Documentary,Film & Video,USD,2014-02-22,10000.0,2014-01-15 06:50:20,471.0,failed,13,US,471.00,471.00,10000.00,0,1
355283,879790563,She Is... Bonni Dance... The new CD is on its ...,Music,Music,USD,2012-09-30,5000.0,2012-08-21 08:13:38,766.0,failed,10,US,766.00,766.00,5000.00,0,1
303932,617571133,Dumbelle ...Pretty Smart! A store for girls ...,Crafts,Crafts,USD,2011-12-08,50000.0,2011-11-08 19:24:10,1305.0,failed,20,US,1305.00,1305.00,50000.00,0,1
333369,768158555,Unforgivable,Shorts,Film & Video,GBP,2014-06-08,6000.0,2014-05-08 18:47:34,6010.0,successful,73,GB,10200.62,10194.22,10177.25,1,0
315192,675402864,EVA: A Digital Safety Net for Drivers Everywhere,Apps,Technology,USD,2014-12-31,300000.0,2014-12-01 18:39:31,107.0,failed,3,US,107.00,107.00,300000.00,0,1
257706,380336998,Ginger KICK! Holiday Cheer!,Small Batch,Food,USD,2017-11-12,50.0,2017-11-05 21:33:24,825.0,successful,23,US,639.00,825.00,50.00,1,1
91710,1465904374,Descent into Hell – God’s Army,Graphic Novels,Comics,USD,2016-11-09,17000.0,2016-10-10 20:37:43,145.0,failed,6,US,0.00,145.00,17000.00,0,1
110170,1559789335,"Sailor Twain, or the Mermaid in the Hudson",Comics,Comics,USD,2011-07-04,7000.0,2011-06-02 12:46:28,7820.0,successful,93,US,7820.00,7820.00,7000.00,1,1


In [29]:
kssample['backers'].nunique()

10

Next, create a linear interpolation of `usd_pledged_real` as a function of the `backers`. Create a function `f` that generates a linear interpolation of `usd_pledged_real` as predicted by the amount of `backers`.

In [47]:
#Create a function f that generates a linear interpolation of usd_pledged_real as predicted by the amount of backers
def f (predict):
    
    X = ks['backers'] 
    Y = ks['usd_pledged_real'] 
  
    for i in ks['backers']:
        interpolate_x = ks['backers'][i]
        y_interp = interp1d(X, Y) #calculate interpolation
        print("When backers is ".format(interpolate_x), "value of usd_pledged_real", y_interp(interpolate_x)) 
        
    return (predict)

In [46]:
f (predict) #calling the function

NameError: name 'predict' is not defined

Now create a new variable called `x_new`. This variable will contain all integers between the minimum number of backers in our sample and the maximum number of backers. The goal here is to take the dataset that contains few obeservations due to sampling and fill all observations with a value using the interpolation function. 

Hint: one option is the `np.arange` function.

In [43]:
#???????????????????????????
x_new=np.arange(ks['backers'])



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Plot function `f` for all values of `x_new`. Run the code below.

In [41]:
# Run this code:

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(x_new, f(x_new))

NameError: name 'x_new' is not defined

Next create a function that will generate a cubic interpolation function. Name the function `g`.

In [ ]:
# Your code here:
def g (cubicstuff):

#cub=st.interpolate.CubicSpline(xxx)
return cubicstuff)

In [ ]:
g(cubicstuff) #calling the function

In [ ]:
# Run this code:

plt.plot(x_new, g(x_new))

# Bonus Challenge - The Binomial Distribution

The binomial distribution allows us to calculate the probability of k successes in n trials for a random variable with two possible outcomes (which we typically label success and failure).  

The probability of success is typically denoted by p and the probability of failure is denoted by 1-p.

The `scipy.stats` submodule contains a `binom` function for computing the probabilites of a random variable with the binomial distribution. You may read more about the binomial distribution [here](http://b.link/binomial55)

* In the cell below, compute the probability that a dice lands on 5 exactly 3 times in 8 tries.


In [ ]:
# Your code here:



* Do a simulation for the last event: do a function that simulate 8 tries and return a 1 if the result is 5 exactly 3 times and 0 if not. Now launch your simulation.

In [ ]:
# Your code here:


* Launch 10 simulations and represent the result in a bar plot. Now launch 1000 simulations and represent it. What do you see?

In [ ]:
# Your code here:
